In [1]:
from tog.dataset_utils.preprocessing import letterbox_image_padded
from keras import backend as K
from tog.models.yolov3 import YOLOv3_Darknet53_Face
from tog.tog.attacks import *
import tensorflow as tf

K.clear_session()
global graph
graph = tf.get_default_graph()

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
session = tf.Session(config=tf_config)


weights = "tog/model_weights/yolo_face.h5"
detector = YOLOv3_Darknet53_Face(weights=weights)



Using TensorFlow backend.
/home/ec2-user/anaconda3/envs/para/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ec2-user/anaconda3/envs/para/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ec2-user/anaconda3/envs/para/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ec2-user/anaco

1


In [2]:
# 모든 사이즈 공격 가능
def attack(input):
    w, h = input.size
    newsize = (w + 416, h + 416)
    new_image = Image.new("RGB", newsize, (0, 0, 0))
    for x in range(0, w, 416):
        for y in range(0, h, 416):
            area = (x, y, x + 416, y + 416)
            cropped_img = input.crop(area)
            rgb = Image.new("RGB", (416, 416), (0, 0, 0))
            rgb.paste(cropped_img, (0, 0))
            cropped_noise = attack416(rgb)
            new_image.paste(cropped_noise, (x, y))
    area = (0, 0, w, h)
    output = new_image.crop(area)
    return output


# 416*416인 이미지만 공격가능
def attack416(input):
    eps = 8 / 255.0
    eps_iter = 2 / 255.0
    n_iter = 10
    npimg = np.asarray(input)[np.newaxis, :, :, :] / 255.0
    with graph.as_default():
        x_adv_untargeted = tog_untargeted(
            victim=detector, x_query=npimg, n_iter=n_iter, eps=eps, eps_iter=eps_iter
        )
    img = x_adv_untargeted[0] * 255
    output = Image.fromarray(img.astype("uint8"), "RGB")
    return output

In [21]:
# !pip3 install opencv-python

src.mp4


In [3]:
import numpy as np
import cv2
from PIL import Image
%matplotlib inline
video_file_path = "video/src.mp4"  # 최종은 input_video로.
cap = cv2.VideoCapture(video_file_path)
# For VideoWriter
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
frame_w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frame_h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
FPS = cap.get(cv2.CAP_PROP_FPS)
untarget_out = cv2.VideoWriter("video/src_attacked.mp4", fourcc, FPS, (int(frame_w), int(frame_h)))

i = 0
while cap.isOpened():e
    ret, frame = cap.read()  # ret = T/F , frame = ndarray(shape: (256,256,3))
    if ret:
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        frame = Image.fromarray(frame)
        untargetedFrame = attack(frame)  # 이미지 반전(0:상하) -> adv 어택.

        untargetedFrame = np.asarray(untargetedFrame)
        untargetedFrame = untargetedFrame[:, :, ::-1]
        # cv2.imshow("asdf",np.asarray(untargetedFrame))
        # cv2.waitKey() 
        # cv2.destroyAllWindows()
        if i % 15 == 0:
            print(f'{i}/{frame_count}')
        i+=1
        untarget_out.write(untargetedFrame)
        if cv2.waitKey(25) == ord("q"):
            break
    else:
        break
cap.release()
untarget_out.release()
cv2.destroyAllWindows()

0/345.0
15/345.0
30/345.0
45/345.0
60/345.0
75/345.0
90/345.0
105/345.0
120/345.0
135/345.0
150/345.0
165/345.0
180/345.0
195/345.0
210/345.0
225/345.0
240/345.0
255/345.0
270/345.0
285/345.0
300/345.0
315/345.0
330/345.0


In [9]:
import matplotlib.pyplot as plt